# **Naturwissenschaft - Gehirn - Practical Work**
**Teacher: Ricardo Chavarriaga**

This practice introduces the general steps for EEG processing including
* Re-referencing
* Filtering
* Epoch extraction and averaging

# Installation
First we will install the necessary libraries. We will use the [MNE](https://www.nmr.mgh.harvard.edu/mne/0.14/index.html) library which is specialized in EEG & MEG analysis

You can find several tutorials here: https://www.nmr.mgh.harvard.edu/mne/0.14/tutorials.html


In [ ]:
!pip install mne
import mne
from mne.datasets import sample
from mne.channels import make_standard_montage
from mne.channels.montage import get_builtin_montages

In [ ]:
%matplotlib inline

# Loading a pre-recorded EEG data file

We will use a dataset provided as example by the MNE library. You can find more information [here](https://www.nmr.mgh.harvard.edu/mne/0.14/manual/sample_dataset.html#ch-sample-data).

In the experiment, checkerboard patterns were presented into the left and right visual field, interspersed by tones to the left or right ear. The interval between the stimuli was 750 ms. Occasionally a smiley face was presented at the center of the visual field. The subject was asked to press a key with the right index finger as soon as possible after the appearance of the face.

The data was recorded using equipment that can record Magnetoencephalograhy (MEG) signals in addition to EEG. MEG signals are changes in electromagnetic fields due to neuron activation. We will remove these signals fof further analysis.

In addition, during the experiment, some electrode were used for measuring eye movements. This type of electrodes are referred to as *Electrooculography" (EOG). The signal of EOG channels can be used to identify and remove signal artifacts due to eye movements. We will cover this in the next lecture.

This experiment include six types of *events* which are included inthe EEG file for allowing analysis of event-related potentials (ERPs). EAch of these events is timestamped in the file with a *Trigger code*


| Condition | Trigger code |
|----|---|
| Response to left-ear auditory stimulus |  1 |
| Response to right-ear auditory stimulus | 2 |
| Response to left visual field stimulus | 3 |
| Response to right visual field stimulus | 4 |
| Response to the smiley face | 5 |
| Response triggered by the button press | 32 |


Setup for reading the raw data



In [ ]:
data_path = sample.data_path()
data_path_converted = str(data_path)
raw_fname = data_path_converted + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
event_fname = data_path_converted + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw.set_eeg_reference()  # set EEG average reference

# Question 1
Examine the information returned by the function and report:
* Recording sampling frequency
* Filter cutt-off frequencies
* Recording duration
* Referencing method for EEG signals
* number of EEG channels
* number of MEG channels
* number of EOG channels



# Channel selection
Here we select the channels that will be used for the analysis. Let's restrict the data to the EEG channels

In [ ]:
raw.pick_types(meg=False, eeg=True, eog=True)

By looking at the measurement info you will see that we have now
59 EEG channels and 1 EOG channel



In practice it's quite common to have some EEG channels that are actually
EOG channels. To change a channel type you can use the
:func:`mne.io.Raw.set_channel_types` method. For example
to treat an EOG channel as EEG you can change its type using



In [ ]:
raw.set_channel_types(mapping={'EOG 061': 'eeg'})
print(raw.info)

And to change the name of the EOG channel



In [ ]:
raw.rename_channels(mapping={'EOG 061': 'EOG'})

Let's reset the EOG channel back to EOG type.



In [ ]:
raw.set_channel_types(mapping={'EOG': 'eog'})

# Plot channel locations
The EEG channels in the sample dataset already have locations.
These locations are available in the 'loc' of each channel description.
For the first channel we get



In [ ]:
print(raw.info['chs'][0]['loc'])

And it's actually possible to plot the channel locations using
the :func:`mne.io.Raw.plot_sensors` method



In [ ]:
raw.plot_sensors()
#raw.plot_sensors('3d')  # Uncomment this line to plot the electrode location in 3D

# Plotting the EEG data
Here, we will plot 10 seconds of the EEG data. Each channel is plotted as a trace line with the signal (in uV) across time.

In [ ]:
raw.plot(duration=10, n_channels=30)

# Plotting the power spectra
Now, we plot the spectrum of the EEG data (i.e. the power at different frequencies) for all channels. We will use the method *Power spectral density* to compute the power at different frequencies. We will study spectral analysis more in detail at the next lecture.

In [ ]:
raw.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

# Filtering
Now, let's filter the signal with a bandpass filter between 4 Hz and 20 Hz and re-plot the frequency spectrum.

In [ ]:
# Band pass filtering between 4 Hz and 30 Hz
filtered = raw.copy().filter(4, 20, h_trans_bandwidth='auto', filter_length='auto',
           phase='zero')

In [ ]:
# plot the the first 10s filtered signal. Compared with the original signal
filtered.plot(duration=10, n_channels=30)

In [ ]:
# plot the frequency spectrum. Compare with the original spectrum
filtered.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

# Question 2
Modify the code below to filter the signal between 8 and 12 HZ, and describe the effect in the spectrum and the signal.

Experiment with 3 other filter configurations.

**NOTE**: Some configurations may not be valid as the function for the digital filter may be unstable or impossible (e.g. high-cut-off < low-cut-off)

In [ ]:
# Band pass filtering between 8 Hz and 12 Hz
filtered = raw.copy().filter(__REPLACE_WITH_LOW_CUTOFF_FREQUENCY_HERE___,__REPLACE_WITH_HIGH_CUTOFF_FREQUENCY_HERE___, 
                             h_trans_bandwidth='auto', filter_length='auto', phase='zero')

# plot the spectra and the filtered signal for the first 5s
filtered.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)
filtered.plot(duration=5, n_channels=30)

In [ ]:
raw.plot(duration=5, n_channels=30)
filtered.plot(duration=5, n_channels=30)

# plot the spectra and the filtered signal for the first 5s
raw.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)
filtered.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

# Epoch extraction and computation of Event-Related Potentials (ERPs)

Let's first define Epochs and compute an ERP for one of the experimental conditions: the left auditory condition.



In [ ]:
raw_no_ref, _ = mne.set_eeg_reference(raw, [])
reject = dict(eeg=180e-6, eog=150e-6) # This command allows rejecting channels with noisy signal

# The following line defines:
#  The type of event for analysis: response to the left auditory stimuli
#  The duration of the epoch (from 200ms before the event until 500ms after the event
event_id, tmin, tmax = {'left/auditory': 1}, -0.2, 0.5
events = mne.read_events(event_fname)
epochs_params = dict(events=events, event_id=event_id, tmin=tmin, tmax=tmax,
                     reject=reject)

evoked_no_ref = mne.Epochs(raw_no_ref, **epochs_params).average()

# plot the ERP
title = 'EEG Original reference'
evoked_no_ref.plot(titles=dict(eeg=title))

# plot the activity in the scalp at 100ms
evoked_no_ref.plot_topomap(times=[0.1, 0.2, 0.4], size=3.)

Now, let's plot the scalp activity at the peak times in the ERP

In [ ]:
evoked_no_ref.plot_topomap(times='peaks', size=3.)

# Referencing
Now let's see the effect of re-referencing

**Average reference**: This is normally added by default, but can also
be added explicitly.



In [ ]:
raw_car, _ = mne.set_eeg_reference(raw)
evoked_car = mne.Epochs(raw_car, **epochs_params).average()
del raw_car  # save memory

title = 'EEG Average reference'
evoked_car.plot(titles=dict(eeg=title))
evoked_car.plot_topomap(times=[0.1, 0.2, 0.4], size=3.)

**Custom reference**: Now, let's change the reference and use the mean of channels EEG 001 and EEG 002 as a reference.



In [ ]:
raw_custom, _ = mne.set_eeg_reference(raw, ['EEG 001', 'EEG 002'])
evoked_custom = mne.Epochs(raw_custom, **epochs_params).average()
del raw_custom  # save memory

title = 'EEG Custom reference'
evoked_custom.plot(titles=dict(eeg=title))
evoked_custom.plot_topomap(times=[0.1, 0.2, 0.4], size=3.)

# Question 3
Describe the effect of changing the reference type:
* Time of the ERP peaks
* Scalp location of maximal and minimal activity (brain area, lateralization)

# Question 4
Change the code below to select a different channel as reference and describe the effect in the signal. EEG channels are labelled 'EEG 001' to 'EEG 060'.

How can you identify which channel was used as reference?

**Note**: Channel 'EEG 053' was faulty during the recording and is labelled as bad channel

In [ ]:
raw_custom, _ = mne.set_eeg_reference(raw, ['__REPLACE_WITH_CHANNEL_NAME'])
evoked_custom = mne.Epochs(raw_custom, **epochs_params).average()
del raw_custom  # save memory

title = 'EEG Custom reference'
evoked_custom.plot(titles=dict(eeg=title))
evoked_custom.plot_topomap(times=[0.1, 0.2, 0.4], size=3.)

# Extracting Epochs for multiple conditions


Now, let's extract and plot ERPs for four experimental conditions:

| Condition | Trigger code |
|----|---|
| left/auditory |  1 |
| right/auditory | 2 |
| left/visual | 3 |
| right/visual | 4 |


Trial subsets from Epochs can be selected using 'tags' separated by '/'.
First, we create an Epochs object containing 4 conditions.

In [ ]:
event_id = {'left/auditory': 1, 'right/auditory': 2,
            'left/visual': 3, 'right/visual': 4}
epochs_params = dict(events=events, event_id=event_id, tmin=tmin, tmax=tmax,
                     reject=reject)
epochs = mne.Epochs(raw, **epochs_params)

Now, let's see how many epochs do we get for each condition

In [ ]:
print(epochs)

# Extract ERPs for stimuli on the left and on the right


Next, we compute the averages of stimulation-left vs stimulation-right trials.


We can use basic arithmetic to, for example, construct and plot
difference ERPs. Here we compute the difference between the average ERP for left condition minus the right condition for each channel.



In [ ]:
left, right = epochs["left"].average(), epochs["right"].average()

# create and plot difference ERP
mne.combine_evoked([left, -right], weights='equal').plot_joint()

# Extracting ERPs for the four conditions






In [ ]:
# We wil now store on the python list 'all_evokeds' all the trials for the conditions listed in variable event_id
event_id = {'left/auditory': 1, 'right/auditory': 2,
            'left/visual': 3, 'right/visual': 4}

# If Evokeds objects are stored in a dictionary, they can be retrieved by name.
all_evokeds = dict((cond, epochs[cond].average()) for cond in event_id)

# Besides for explicit access, this can be used for example to set titles.
for cond in all_evokeds:
    all_evokeds[cond].plot_joint(title=cond)


# Question 5

Describe the differences in the ERPs obtained for each condition in terms of:
* Time of the ERP peaks
* Scalp location of maximal and minimal activity (brain area, lateralization)
* Hypothesize on which brain areas may be involved in each condition. You can find a representation of the different brain lobes [here](https://1.bp.blogspot.com/-ZsryFerSXdQ/X-ZNzFwHytI/AAAAAAAAKWk/nZQbiZZ3aZoe7O5rG1hs8A_68XFNyJnHwCLcBGAsYHQ/s1600/parts%2Bbrain.jpg)
